In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# load source dataset containing point coordinates
df = gpd.read_file('raw/ladesaeulen_standorte.geojson')

# we want to find the respective plr for each point coordinate, so we need the plr polygons
plr = gpd.read_file('../plr/plr_only.geojson')

# plr without geometry (for merging later on)
plr_no_geo = plr.drop(columns='geometry').sort_values(by='PLR_ID')

# check for same crs (coordinate reference system)
df.crs == plr.crs

True

In [3]:
# Spatial join assigns each point to its respective plr
df_with_plr = gpd.sjoin(df, plr, how='inner', predicate='within')

print("df shape:", df.shape, "\ndf_with_plr shape:", df_with_plr.shape)

df shape: (2197, 22) 
df_with_plr shape: (2197, 25)


In [4]:
df_with_plr.head()

,importid,standort_nr,plz,ort,adresse,bezirk,betreiber,inbetriebnahme,art_ladeeinrichtung,anschluss_1,...,ladeleistung_3,ladeleistung_4,anzahl_ladepunkte,oeffentlicher_raum,datenquelle,symbol,geometry,index_right,PLR_ID,PLR_NAME
0,1,DEQRMEM9J3C11,13359,Berlin,Holzstraße 6,Mitte,Qwello GmbH,2023-06-20 22:00:00+00:00,AC,AC Steckdose Typ 2,...,NaN,NaN,1,ja,Stadt Berlin,öffentlicher Raum,MULTIPOINT (390136.000 5824753.500),474,01300731,Koloniestraße
1,2,DEQRMEM9J3C21,13359,Berlin,Holzstraße 6,Mitte,Qwello GmbH,2023-06-20 22:00:00+00:00,AC,AC Steckdose Typ 2,...,NaN,NaN,1,ja,Stadt Berlin,öffentlicher Raum,MULTIPOINT (390136.000 5824753.500),474,01300731,Koloniestraße
1892,1893,1046478,13359,Berlin,Nordbahnstr. 5,Mitte,Berliner Stadtwerke KommunalPartner GmbH,2020-10-06 22:00:00+00:00,AC,AC Steckdose Typ 2,...,NaN,NaN,2,ja,Stadt Berlin,öffentlicher Raum,MULTIPOINT (390995.781 5825118.029),474,01300731,Koloniestraße
2131,2132,1260722,13359,Berlin,Soldiner Str. 63,Mitte,Berliner Stadtwerke KommunalPartner GmbH,2023-05-30 22:00:00+00:00,AC,AC Steckdose Typ 2,...,NaN,NaN,2,ja,Stadt Berlin,öffentlicher Raum,MULTIPOINT (390142.299 5824618.686),474,01300731,Koloniestraße
2,3,NaN,10963,Berlin,Gabriele-Tergit-Promenade 19,Mitte,Scandic Hotels Deutschland GmbH,2019-07-02 22:00:00+00:00,AC,AC Steckdose Typ 2,...,NaN,NaN,1,nein,Stadt Berlin,privater Grund,MULTIPOINT (389729.054 5818551.663),333,01100102,Großer Tiergarten


In [5]:
df_with_plr.columns

Index(['importid', 'standort_nr', 'plz', 'ort', 'adresse', 'bezirk',
       'betreiber', 'inbetriebnahme', 'art_ladeeinrichtung', 'anschluss_1',
       'anschluss_2', 'anschluss_3', 'anschluss_4', 'ladeleistung_1',
       'ladeleistung_2', 'ladeleistung_3', 'ladeleistung_4',
       'anzahl_ladepunkte', 'oeffentlicher_raum', 'datenquelle', 'symbol',
       'geometry', 'index_right', 'PLR_ID', 'PLR_NAME'],
      dtype='object')

In [6]:
# only keep summary relevant features, get rid of geometry -> regular pandas dataframe
feature_list = [
    'PLR_ID', 'anzahl_ladepunkte',
    'ladeleistung_1', 'ladeleistung_2', 'ladeleistung_3', 'ladeleistung_4']
df_with_plr = df_with_plr[feature_list]

# NaN to 0
df_with_plr = df_with_plr.fillna(0)

df_with_plr['ladeleistung_gesamt'] = (df_with_plr['ladeleistung_1']
                                      + df_with_plr['ladeleistung_2']
                                      + df_with_plr['ladeleistung_3']
                                      + df_with_plr['ladeleistung_4'])

df_with_plr = df_with_plr[['PLR_ID', 'anzahl_ladepunkte', 'ladeleistung_gesamt']]
df_with_plr

,PLR_ID,anzahl_ladepunkte,ladeleistung_gesamt
0,01300731,1,11.0
1,01300731,1,11.0
1892,01300731,2,22.0
2131,01300731,2,44.0
2,01100102,1,22.0
...,...,...,...
2178,09100407,1,3.0
2179,09100407,1,3.0
2180,09100407,1,3.0
2181,09100407,1,3.0


In [7]:
# Summarize by groupby
gb = df_with_plr.groupby('PLR_ID')

count = gb.size().reset_index(name='charging_stations')
n_points = gb['anzahl_ladepunkte'].sum().reset_index(name='charging_points')
combined_power = gb['ladeleistung_gesamt'].sum().reset_index(name='combined_power_kW')

In [8]:
# Merge
result = plr_no_geo \
    .merge(count, on='PLR_ID', how='outer') \
    .merge(n_points, on='PLR_ID', how='outer') \
    .merge(combined_power, on='PLR_ID', how='outer')

result.shape

(542, 5)

In [9]:
result.head()

,PLR_ID,PLR_NAME,charging_stations,charging_points,combined_power_kW
0,01100101,Stülerstraße,3.0,6.0,110.0
1,01100102,Großer Tiergarten,35.0,40.0,693.0
2,01100103,Lützowstraße,5.0,13.0,165.0
3,01100104,Körnerstraße,3.0,6.0,66.0
4,01100205,Wilhelmstraße,11.0,16.0,234.0


In [10]:
result.to_csv("charging_stations_per_plr.csv", index=False)